# Model Predictive Control

In [1]:
import numpy as np
np.set_printoptions(precision=2, suppress=True)
from matplotlib import pyplot as plt
import pdb
%matplotlib inline

# 1. magic to print version
# 2. magic so that the notebook will reload external python modules
# %load_ext watermark
%load_ext autoreload
%autoreload 2

## Model equations

 - 고려해야할 요소
1. Gravity, Inertia
2. Road surface friction
3. Vehicle wieght variation
  
 - state variables  
$(x, y, \psi, v, cte, e\psi)$.로 6 가지 변수로 구성된다.  
Cross Track Error ($cte$), Heading angle error ($e\psi$)
  
 - kinematics and dynamic mathematical model equations 에 근거한 system dynamics는 아래와 같이 표현된다.   
  
$$
\begin{align}
X(t+1) &= X(t) + V(t) \;cos(\psi(t) ) \;dt \\
Y(t+1) &= Y(t) + V(t) \;sin(\psi(t) ) \;dt \\
\psi(t+1) &= \psi(t) + \frac{V(t)}{Lf} \; \delta(t) \; dt \\
V(t+1) &= V(t) + a(t) \; dt \\
cte(t+1) &= cte(t) + \Delta Y(t)  \\
&= f(x(t)) - Y(t) + V(t) \; sin(\psi(t)) \;dt \\
e\psi(t+1) &= e\psi(t) + \Delta \psi(t) \\
&= \psi(t) - \nabla f(x(t)) + \frac{V(t)}{Lf} \; \delta(t) \; dt 
\end{align}
$$

<img src="https://static.wixstatic.com/media/850e97_29716bdc4dd9433099e5bf002d54952c~mv2.png/v1/fill/w_600,h_348,al_c,q_85,usm_0.66_1.00_0.01/850e97_29716bdc4dd9433099e5bf002d54952c~mv2.webp" width=500>
  
 - 자동차는 nonholonomic system 이기 때문에 action에 다음과 같은 physical constraint를 갖는다.
  
$$
\begin{align}
- 40^{\circ} < \; &\delta \;< 40^{\circ} \\
- 1 < \; &Throttle \;< 1 \\
0 < \; &Speed \;< speedlimit
\end{align}
$$

## Trajectory costs
  
주어진 trajectory를 시뮬레이션 해보고 cost를 계산해서 가장 작은 cost를 갖는 actuator의 command를 찾아야한다.  
cost 계산에는 아래와 같은 3가지 규칙이 있다.
 
1. desired state와 current state간의 차이
$$
\sum_{t=1}^{N} w_{cte} \;cte^{2}_{t} + w_{e\psi} \;e\psi^{2}_{t} + w_{v} \;(V_{t} - V_{ref})^{2}
$$
2. throttle 과 steering 의 사용 
$$
\sum_{t=1}^{N} w_{\delta} \; \delta^{2}_{t} + w_{a} \; a^{2}_{t}
$$
3. throttle 과 steering 의 변화량
$$
\sum_{t=1}^{N} w_{\Delta \delta} \; (\delta_{t+1} -\delta_{t})^{2} + w_{\Delta a} \; (a_{t+1} - a_{t})^{2}
$$

### System parameter
총 system parameter 는 9가지로 다음과 같다.  
penalty에 대한 weight 은 7가지이다.  
  
$$
\bigg[w_{cte}, w_{e\psi}, w_{v},  w_{\delta},  w_{a}, w_{\Delta \delta}, w_{\Delta a}\bigg]
$$
  
그리고 나머지 2가지 N 과 dt는 다음과 같은 성질을 가진다. 
  
 - N 이 작다면 short term planning 이기 때문에 overshooting과 instability에 문제가 발생한다.
 - N 이 크다면 computational cost가 증가하기 때문에 FPS가 증가하여 대응이 늦어진다.
 - dt 가 크다면 필요한 control command가 늦어지기 때문에 대응이 늦어진다.
 - dt 작다면 actuator가 반응하기 전에 control이 주어지기 때문에 flunctuation 이 발생한다.
  
system settling time (system latency) 가 약 100ms가 걸린다고 했을 때 dt는 이 보다 더 크게 설정해야하기 때문에 120ms 정도로 설정한다.  
그리고 desired speed가 30km/h (8.3m/s) 라고 생각 했을 때 약 10m 정도 앞까지 cost를 고려하려면 
$$
N = \frac{10}{8.3 \times 0.12} \approx 10
$$
으로 설정하면 적절하다.

## MPC process
  
다음과 같은 과정으로 진행된다.

1. trajectory paths 를 the vehicle-centric reference frame 으로 바꾼다.
    > subscribe 한 경로 좌표계는 오른손 좌표이므로 변환할 필요 없고, 자차의 위치및 heading angle은 오른손좌표로 변환 필요.
2. polynormial function 으로 주어진 path를 fitting힌다.
    > 자차 기준 5미터 전후의 point들을 가져와서 fitting해야 안정적인 fitting이 됨.
3. $cte$ 와 $e\psi$ 를 계산하여 현 스텝에 대한 state error를 구한다.
4. model variable을 초기화하고 각 변수에 constraint 에 맞는 limit 을 할당한다.
    > 무게중심은 대략적으로 전장의 4:6 정도 비율을 갖음 이값을 기준으로 값을 바꿔가면서 oversteering 이나 understeering이 나지 않도록 조정.
5. system constraint와 physics constriant 를 고려하여 tracjectory cost를 optimize 하는 N step의 control command를 구한다.
6. optmize한 첫번째 command를 실행한다.

## Reference  
  
[MPC](https://www.haidynmcleod.com/model-predictive-controller)  
[Lateral Dynamic Equations of Vehicle](https://m.blog.naver.com/jjz0426/221135413776)  
[Path tracking algorithm 비교](https://www.ri.cmu.edu/pub_files/2009/2/Automatic_Steering_Methods_for_Autonomous_Automobile_Path_Tracking.pdf)